In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df = pd.read_csv('/Users/mgor/Downloads/IMDB_Top250Engmovies2_OMDB_Detailed.csv')

df1 = df[['ID1','Title','imdbRating']]
df1 = df1.to_dict('ID1')
# print(df1)

df2 = df[['Title','Genre','Director','Actors','Plot']]
df2.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [2]:
# initializing the new column
df2['Key_words'] = ""
df2['bag_of_words'] = ""

for index, row in df2.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())
    row['bag_of_words'] = ' '.join(list(key_words_dict_scores.keys()))

# dropping the Plot column
df2.drop(columns = ['Plot'], inplace = True)

/Users/mgor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mgor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mgor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.py

In [3]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df2['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [4]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df2.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [5]:
movie_id = 3
recommended_movies = recommendations(movie_id)
print(recommended_movies)

[51, 203, 215, 36, 29, 89, 219, 9, 187, 45]


In [6]:
sum_rating = 0
total_rating = 0

for i in range(len(recommended_movies)):
    if i < 5:
        sum_rating += df1[recommended_movies[i-1]]['imdbRating']*5
        total_rating += 5
    elif i < 7:
        sum_rating += df1[recommended_movies[i-1]]['imdbRating']*3
        total_rating += 3
    else:
        sum_rating += df1[recommended_movies[i-1]]['imdbRating']
        total_rating += 1
    print(df1[recommended_movies[i-1]]['Title'])

final_rating = sum_rating/total_rating
print(final_rating)
print('\nFinal rating of the new movie : ' + df1[recommended_movies[movie_id-1]]['Title'] + ' is --> ' + str(final_rating))

Alien
The Dark Knight Rises
Guardians of the Galaxy Vol. 2
Sin City
Terminator 2: Judgment Day
The Green Mile
Batman Begins
The Avengers
The Lord of the Rings: The Fellowship of the Ring
Out of the Past
8.33529411764706

Final rating of the new movie : Sin City is --> 8.33529411764706
